<a href="https://colab.research.google.com/github/Decoding-Data-Science/nov25/blob/main/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a great request. Drawing on the uses cases mentioned in the video sources, I will focus on the example of **e-commerce product recommendations** and generate synthetic data and corresponding TensorFlow/Keras code.

The sources highlight that TensorFlow is an open-source library created by Google used to **build, train, and deploy** machine learning models at a massive scale. For beginners or general developers, the recommended path is using **Keras**, the user-friendly, high-level API built right on top of TensorFlow. The Keras workflow is simply three steps: define, compile, and train.

## 1. Synthetic Large Data Creation: E-commerce Product Recommendations

E-commerce sites use TensorFlow for product recommendations. To model this, we will create a dataset representing user interactions (ratings) with products. Since the query asks for "synthetic large data," we will generate 100,000 interaction records.

*Note: While the source material focuses on TensorFlow and Keras, standard Python libraries like `numpy` and `pandas` (which are not explicitly mentioned in the sources) are required to generate and structure this large synthetic dataset.*

### Use Case: Simplified Collaborative Filtering Model

We simulate data based on:
1.  **Number of Users:** 1,000
2.  **Number of Products:** 500
3.  **Total Interactions (Ratings):** 100,000


In [1]:

import numpy as np
import pandas as pd

# Define parameters for 'large' synthetic data
NUM_USERS = 1000
NUM_PRODUCTS = 500
NUM_INTERACTIONS = 100000

# 1. Create Synthetic IDs
# Generate a large array of random User IDs
user_ids = np.random.randint(1, NUM_USERS + 1, NUM_INTERACTIONS)
# Generate a large array of random Product IDs
product_ids = np.random.randint(1, NUM_PRODUCTS + 1, NUM_INTERACTIONS)
# Generate interaction ratings (e.g., 1 to 5 stars)
ratings = np.random.randint(1, 6, NUM_INTERACTIONS)

# 2. Structure the data into a DataFrame
data = pd.DataFrame({
    'user_id': user_ids,
    'product_id': product_ids,
    'rating': ratings
})

# Display the size and a sample of the synthetic data
print(f"--- Synthetic Data for E-commerce Recommendations ---")
print(f"Total Interactions Generated: {len(data)} (Large Scale Data)")
print(data.head())

# 3. Prepare data for model training
# Determine unique counts for embedding layers
unique_users = data['user_id'].nunique()
unique_products = data['product_id'].nunique()

# Map IDs to contiguous integers starting from 0 (required for embeddings)
user_map = {id: i for i, id in enumerate(data['user_id'].unique())}
product_map = {id: i for i, id in enumerate(data['product_id'].unique())}

data['user'] = data['user_id'].map(user_map)
data['product'] = data['product_id'].map(product_map)

# Define X (inputs) and y (target)
X = data[['user', 'product']].values
y = data['rating'].values

# Split data into training and testing sets (standard preparation step)
from sklearn.model_selection import train_test_split # Note: sklearn is not mentioned in sources
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Extract inputs for the model
train_user_input = X_train[:, 0]
train_product_input = X_train[:, 1]


## 2. TensorFlow/Keras Code for the Recommendation Model

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense
from tensorflow.keras.models import Model


--- Synthetic Data for E-commerce Recommendations ---
Total Interactions Generated: 100000 (Large Scale Data)
   user_id  product_id  rating
0      231          27       2
1      597         299       1
2      315          34       5
3      734         245       3
4      408         259       3


In [2]:

# --- Step 1: Define the Model (Stacking layers like Legos) ---

# Define constants based on the synthetic data preparation
EMBEDDING_SIZE = 50 # Dimension for the learned embeddings
NUM_USERS = unique_users # Total number of unique users
NUM_PRODUCTS = unique_products # Total number of unique products

# 1. Input layers
# Define the input layer for User IDs
user_input = Input(shape=(1,), name='user_input')
# Define the input layer for Product IDs
product_input = Input(shape=(1,), name='product_input')

# 2. Embedding layers (The core of collaborative filtering)
# Embedding layer for users
user_embedding = Embedding(
    input_dim=NUM_USERS,
    output_dim=EMBEDDING_SIZE,
    input_length=1,
    name='user_embedding'
)(user_input)

# Flatten the user embedding
user_vec = Flatten(name='user_flatten')(user_embedding)

# Embedding layer for products
product_embedding = Embedding(
    input_dim=NUM_PRODUCTS,
    output_dim=EMBEDDING_SIZE,
    input_length=1,
    name='product_embedding'
)(product_input)

# Flatten the product embedding
product_vec = Flatten(name='product_flatten')(product_embedding)

# 3. Combine and Predict
# Concatenate user and product vectors
concat = Concatenate()([user_vec, product_vec])

# Add Dense layers (fully connected layers) to process the combined vector
dense_1 = Dense(128, activation='relu')(concat)
dense_2 = Dense(64, activation='relu')(dense_1)

# Output layer: Predict the final rating (or interaction score)
# Since ratings are 1-5, we use a sigmoid output scaled to the range 1-5
output = Dense(1, activation='linear')(dense_2)

# Create the final Model object using the inputs and output
model = Model(inputs=[user_input, product_input], outputs=output)


# --- Step 2: Compile the Model (Telling it how to learn) ---

# We choose the Adam optimizer and Mean Squared Error loss (common for regression tasks like rating prediction)
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mae'] # Mean Absolute Error for easier interpretability
)

# Print a summary of the model structure
model.summary()


# --- Step 3: Train the Model (Calling the fit function) ---

print("\nStarting Model Training...")
# Calling fit() gives the data to TensorFlow, which handles the heavy lifting
history = model.fit(
    x=[train_user_input, train_product_input], # Training input data
    y=y_train,                              # Training target data (ratings)
    epochs=5,                               # Number of training iterations
    batch_size=256,
    verbose=1,
    validation_split=0.1                    # Hold back 10% of training data for validation
)

print("Model Training Complete.")

# --- Post-Training Step: Prepare for Deployment ---
# Once trained and validated (perhaps using TensorBoard for visualization),
# the model can be optimized and prepared for deployment on devices using TFLite.
# model.save('recommendation_model.h5')
# # Then convert to TFLite format for efficient device deployment:
# # converter = tf.lite.TFLiteConverter.from_keras_model(model)
# # tflite_model = converter.convert

c:\Users\almehairbi\Desktop\Dell AI\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ product_input       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 1, 50)     │     50,000 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ product_embedding   │ (None, 1, 50)     │     25,000 │ product_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_flatten        │ (None, 50)        │          0 │ user_embedding[0… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ product_flatten     │ (None, 50)        │          0 │ product_embeddin… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 100)       │          0 │ user_flatten[0][… │
│ (Concatenate)       │                   │            │ product_flatten[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     12,928 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         65 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 96,249 (375.97 KB)

 Trainable params: 96,249 (375.97 KB)

 Non-trainable params: 0 (0.00 B)


Starting Model Training...
Epoch 1/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 2.8034 - mae: 1.3882 - val_loss: 2.0171 - val_mae: 1.2234
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2.0029 - mae: 1.2189 - val_loss: 2.0193 - val_mae: 1.2236
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.9856 - mae: 1.2136 - val_loss: 2.0154 - val_mae: 1.2204
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.9768 - mae: 1.2120 - val_loss: 2.0188 - val_mae: 1.2232
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.9566 - mae: 1.2063 - val_loss: 2.0456 - val_mae: 1.2339
Model Training Complete.


In [3]:
# --- Step 4: Final Evaluation on Unseen Test Data ---

print("\n--- Model Evaluation on Unseen Test Set ---")

# Extract inputs for the model from X_test
test_user_input = X_test[:, 0]
test_product_input = X_test[:, 1]

# The evaluation process assesses the performance of the trained model
# on the independent dataset (X_test, y_test) that the model has never seen.
results = model.evaluate(
    x=[test_user_input, test_product_input], # Test input data
    y=y_test,                                # Test target data
    batch_size=256,
    verbose=0
)

# Extract and display the metrics defined during the compile step
loss = results[0]
mae = results[1] # Mean Absolute Error (the chosen evaluation metric)

print(f"Test Loss (Mean Squared Error): {loss:.4f}")
print(f"Test Mean Absolute Error (MAE): {mae:.4f}")


--- Model Evaluation on Unseen Test Set ---
Test Loss (Mean Squared Error): 2.0594
Test Mean Absolute Error (MAE): 1.2405


### Enhancing Understanding

TensorFlow provides an entire ecosystem. If we were to continue this workflow:

*   **Visualization:** While the model is training, we would typically use **TensorBoard** to visualize the accuracy and spot problems, acting as a dashboard to see the model's mind at work.
*   **Deployment:** Once training is complete, the model would be optimized using **TensorFlow Lite (TFLite)**. TFLite specializes in making the powerful model super small and efficient so it can run on edge devices like smartphones or smart home gadgets, allowing the AI to run right on the device for speed and privacy.

The Keras workflow—defining layers, compiling, and calling `fit`—is like using **Lego bricks** to build a structure. The Lego bricks are the layers (Input, Embedding, Dense), and the compiled model is the instruction manual telling the structure how to stand up and function (the optimizer and loss function). When you call `fit`, you are simply starting the building process with the training data.